In [1]:
#https://google.github.io/mediapipe/

import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

# For static images:
IMAGE_FILES = []
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
with mp_face_mesh.FaceMesh(
    static_image_mode=True,
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5) as face_mesh:
  for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    # Convert the BGR image to RGB before processing.
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Print and draw face mesh landmarks on the image.
    if not results.multi_face_landmarks:
      continue
    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
      print('face_landmarks:', face_landmarks)
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_TESSELATION,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_tesselation_style())
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_CONTOURS,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_contours_style())
      mp_drawing.draw_landmarks(
          image=annotated_image,
          landmark_list=face_landmarks,
          connections=mp_face_mesh.FACEMESH_IRISES,
          landmark_drawing_spec=None,
          connection_drawing_spec=mp_drawing_styles
          .get_default_face_mesh_iris_connections_style())
    cv2.imwrite('/tmp/annotated_image' + str(idx) + '.png', annotated_image)
    
# For webcam input:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
cap = cv2.VideoCapture(0)
with mp_face_mesh.FaceMesh(
    max_num_faces=1,
    refine_landmarks=True,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as face_mesh:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(image)

# https://www.youtube.com/watch?v=Yg6bFRnOSbs

    # Draw the face mesh annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_face_landmarks:
      for face_landmarks in results.multi_face_landmarks:
        #if face_landmarks in mp_face_mesh.FACEMESH_LIPS:
        mp_drawing.draw_landmarks(
            image=image,
            landmark_list=face_landmarks,
            connections=mp_face_mesh.FACEMESH_TESSELATION,
            landmark_drawing_spec=None,
            connection_drawing_spec=mp_drawing_styles
            .get_default_face_mesh_tesselation_style())
        #mp_drawing.draw_landmarks(
        #    image=image,
        #    landmark_list=face_landmarks,
        #    connections=mp_face_mesh.FACEMESH_CONTOURS,
        #    landmark_drawing_spec=None,
        #    connection_drawing_spec=mp_drawing_styles
        #    .get_default_face_mesh_contours_style())
        #mp_drawing.draw_landmarks(
        #    image=image,
        #    landmark_list=face_landmarks,
        #    connections=mp_face_mesh.FACEMESH_IRISES,
        #    landmark_drawing_spec=None,
        #    connection_drawing_spec=mp_drawing_styles
        #    .get_default_face_mesh_iris_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Face Mesh', cv2.flip(image, 1))
    if cv2.waitKey(5) & 0xFF == 27:
      break
cap.release()
cv2.destroyAllWindows()

In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
#mp_drawing_styles = mp.solutions.drawing_styles
#mp_face_mesh = mp.solutions.face_mesh
# https://www.youtube.com/watch?v=mCcPmlr7y3U&t=2s

webcam = cv2.VideoCapture(0)
sol_rec_face = mp.solutions.face_detection
rec_face = sol_rec_face.FaceDetection()
desenho = mp.solutions.drawing_utils

print("start tutorial")

while True:
    verify, frame =  webcam.read()

    if not verify:
        break

    face_list = rec_face.process(frame)

    if face_list.detections:
        for face in face_list.detections:
            desenho.draw_detection(frame, face)

    cv2.imshow("faces", frame)

    # quando aperta ESC finaliza
    if cv2.waitKey(5) == 27:
        break

webcam.release()
cv2.destroyAllWindows()

start tutorial


In [2]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh
#mp_drawing = mp.solutions.drawing_utils
#mp_drawing_styles = mp.solutions.drawing_styles
#mp_face_mesh = mp.solutions.face_mesh
# https://www.youtube.com/watch?v=mCcPmlr7y3U&t=2s

#mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

image = cv2.imread('sorriso.jpeg')
rgb_image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)

height, width, _ = image.shape

results = face_mesh.process(rgb_image)

#type(results.multi_face_landmarks)
#results.multi_face_landmarks

#mp_face_mesh.FACEMESH_LIPS

facial_landmarks = results.multi_face_landmarks[0]
#for facial_landmarks in results.multi_face_landmarks:
#    pt1 = facial_landmarks.landmark[0]
#    x = int(pt1.x * width)
#    y = int(pt1.y * height)
#    cv2.circle(image, (x,y), 7 ,(100,100,0), -1)

for source_idx, target_idx in mp_face_mesh.FACEMESH_LIPS:

    source = facial_landmarks.landmark[source_idx]
    target = facial_landmarks.landmark[target_idx]

    relative_source = (int(source.x * image.shape[1]), int(source.y * image.shape[0]))
    relative_target = (int(target.x * image.shape[1]), int(target.y * image.shape[0]))

    cv2.line(image, relative_source,relative_target, color= (255,255,255), thickness= 2)

cv2.imshow("Image",image)
#cv2.imshow("Image",rgb_image)

cv2.waitKey(0)

cv2.destroyAllWindows()

In [10]:
# Standard imports
import cv2
import numpy as np 

# https://learnopencv.com/seamless-cloning-using-opencv-python-cpp/
# Read images
src = cv2.imread("airplane.jpg")
#dst = cv2.imread("clouds.jpg")
dst = cv2.imread("sky.jpg")

print(src.shape)
print(dst.shape)

# Create a rough mask around the airplane.
src_mask = np.zeros(src.shape, src.dtype)
poly = np.array([ [4,80], [30,54], [151,63], [254,37], [298,90], [272,134], [43,122] ], np.int32)
cv2.fillPoly(src_mask, [poly], (255, 255, 255))

# This is where the CENTER of the airplane will be placed
center = (800,200)

# Clone seamlessly.
output = cv2.seamlessClone(src, dst, src_mask, center, cv2.NORMAL_CLONE)

# Save result
cv2.imwrite("opencv-seamless-cloning-example.jpg", output);

(194, 300, 3)
(561, 1000, 3)
